<a href="https://colab.research.google.com/github/thedavidneufeld/ResearchCode/blob/main/RQI_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [ ]:
import numpy as np

**Helper Functions**

In [ ]:
# given an approximate eigenvalue (mu) and a matrix M
# returns (mu)*I-M
def A(mu, M):
    I = np.eye(M.shape[0])
    return mu*I-M

# calculates and returns the conditon number of M
# based on sigma_1/sigma_n
def svd_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-1]

# calculates and returns sigma_1/sigma_{n-1}
def svd_desired_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-2]

# calculates a preconditioned matrix C based on
# an approximate eigenvalue (mu), a matrix M, and vectors
# y and v
# returns C
def f(mu, M, y, v):
    # convert y and v to column vectors
    Y = y[:, np.newaxis]
    V = v[:, np.newaxis]
    # calculate (mu)*I-M
    A1 = A(mu, M)
    # calculate ((mu)*I-M)/||((mu)*I-M)||+YV^H
    C = A1/np.linalg.norm(A1, 2) + np.dot(Y, V.T.conj())
    # return C
    return C

**RQI Algorithm**

In [ ]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
def RQI(M, mu, ep, it):
    # create a random vector with norm of 1
    # vector needs to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    # track condition number of (mu)I-M through iterations
    conds = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate the condition number of (mu)I-M
        conds = np.append(conds, svd_cond(muI_M))
        # solve the system ((mu)I-M)x=y
        x = np.linalg.solve(muI_M, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (x^T)Mx
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x, 2) <= ep*np.linalg.norm(M, 2):
            return mu, y, i, conds
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds


**RQI Algorithm with Additive Preconditioning**

In [ ]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
#
# similar to regular RQI, but uses additive preprocessing
def RQIAPP(M, mu, ep, it):
    # create two random vectors with norm of 1
    # vectors need to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    v = np.random.rand(M.shape[0])
    v = v/np.linalg.norm(v)
    # track condition number of (mu)I-M, C, and the desired condition number
    conds = np.empty(0)
    conds_C = np.empty(0)
    conds_desired = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate C
        C = f(mu, M, y, v)
        # calculate the condition numbers
        conds = np.append(conds, svd_cond(muI_M))
        conds_C = np.append(conds_C, svd_cond(C))
        conds_desired = np.append(conds_desired, svd_desired_cond(muI_M))
        # solve the system Cx=y
        x = np.linalg.solve(C, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (y^T)My
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x, 2) <= ep*np.linalg.norm(M, 2):
            return mu, y, i, conds, conds_C, conds_desired
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds, conds_C, conds_desired

**Test and Compare Algorithms**

In [ ]:
# Setup

# create a random matrix M
M = np.random.rand(100, 100)
# calculate the eigenvalues/eigenvectors of M
vals, vecs = np.linalg.eig(M)
# extract the first eigenvalue of M and subtract
# a small value to make it an approximation
mu = vals[0]-0.25

In [ ]:
# Print condition number of M
svd_cond(M)

8846.596445838837

In [ ]:
# Test RQI with 1e-4
result = RQI(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (49.48881551338604+0j)
Eigenvector: [0.0883701 +0.j 0.10441201+0.j 0.10975579+0.j 0.1051238 +0.j
 0.09593866+0.j 0.09503691+0.j 0.10493568+0.j 0.0979904 +0.j
 0.09799768+0.j 0.09722706+0.j 0.09886003+0.j 0.09245574+0.j
 0.10143199+0.j 0.09972555+0.j 0.09984458+0.j 0.09052965+0.j
 0.08862268+0.j 0.09206534+0.j 0.09732394+0.j 0.11534046+0.j
 0.10293405+0.j 0.10598363+0.j 0.0888036 +0.j 0.09633756+0.j
 0.10660603+0.j 0.09291242+0.j 0.10249836+0.j 0.10772202+0.j
 0.09376061+0.j 0.10160192+0.j 0.1086145 +0.j 0.09839232+0.j
 0.0927464 +0.j 0.10433217+0.j 0.10455707+0.j 0.09719659+0.j
 0.0938102 +0.j 0.10690811+0.j 0.10017636+0.j 0.09984142+0.j
 0.09813852+0.j 0.0975342 +0.j 0.10444276+0.j 0.10495577+0.j
 0.0964888 +0.j 0.09819335+0.j 0.10830213+0.j 0.10247659+0.j
 0.10436206+0.j 0.09805368+0.j 0.10896403+0.j 0.09926827+0.j
 0.0946917 +0.j 0.10575159+0.j 0.09129066+0.j 0.09871293+0.j
 0.0963098 +0.j 0.10508999+0.j 0.09243135+0.j 0.08626876+0.j
 0.09801795+0.j 0.11475194+0.j 0.0980

In [ ]:
# Test RQIAPP with 1e-4
result = RQIAPP(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers for (mu)I-M:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.48881550519574+0j)
Eigenvector: [0.0883701 +0.j 0.10441202+0.j 0.1097558 +0.j 0.10512379+0.j
 0.09593866+0.j 0.09503691+0.j 0.10493568+0.j 0.09799041+0.j
 0.09799769+0.j 0.09722706+0.j 0.09886003+0.j 0.09245574+0.j
 0.10143198+0.j 0.09972555+0.j 0.09984457+0.j 0.09052966+0.j
 0.08862269+0.j 0.09206533+0.j 0.09732394+0.j 0.11534046+0.j
 0.10293405+0.j 0.10598363+0.j 0.0888036 +0.j 0.09633755+0.j
 0.10660604+0.j 0.09291241+0.j 0.10249837+0.j 0.10772202+0.j
 0.09376062+0.j 0.10160192+0.j 0.10861449+0.j 0.09839231+0.j
 0.0927464 +0.j 0.10433217+0.j 0.10455707+0.j 0.09719659+0.j
 0.09381019+0.j 0.1069081 +0.j 0.10017636+0.j 0.09984142+0.j
 0.09813852+0.j 0.09753421+0.j 0.10444277+0.j 0.10495578+0.j
 0.0964888 +0.j 0.09819336+0.j 0.10830213+0.j 0.1024766 +0.j
 0.10436206+0.j 0.09805368+0.j 0.10896404+0.j 0.09926826+0.j
 0.0946917 +0.j 0.10575159+0.j 0.09129067+0.j 0.09871292+0.j
 0.09630979+0.j 0.10508999+0.j 0.09243134+0.j 0.08626875+0.j
 0.09801796+0.j 0.11475194+0.j 0.0980

In [ ]:
# Test RQI with 1e-6
result = RQI(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (49.4888155022093+0j)
Eigenvector: [-0.0883701 +0.j -0.10441202+0.j -0.10975579+0.j -0.1051238 +0.j
 -0.09593866+0.j -0.09503692+0.j -0.10493568+0.j -0.09799041+0.j
 -0.09799769+0.j -0.09722705+0.j -0.09886004+0.j -0.09245574+0.j
 -0.10143199+0.j -0.09972556+0.j -0.09984457+0.j -0.09052966+0.j
 -0.08862269+0.j -0.09206533+0.j -0.09732395+0.j -0.11534046+0.j
 -0.10293405+0.j -0.10598362+0.j -0.0888036 +0.j -0.09633755+0.j
 -0.10660604+0.j -0.09291242+0.j -0.10249836+0.j -0.10772202+0.j
 -0.09376062+0.j -0.10160192+0.j -0.10861449+0.j -0.09839232+0.j
 -0.0927464 +0.j -0.10433217+0.j -0.10455707+0.j -0.09719658+0.j
 -0.09381019+0.j -0.1069081 +0.j -0.10017636+0.j -0.09984142+0.j
 -0.09813852+0.j -0.09753421+0.j -0.10444276+0.j -0.10495578+0.j
 -0.09648881+0.j -0.09819336+0.j -0.10830213+0.j -0.10247659+0.j
 -0.10436205+0.j -0.09805367+0.j -0.10896404+0.j -0.09926826+0.j
 -0.0946917 +0.j -0.10575159+0.j -0.09129067+0.j -0.09871292+0.j
 -0.0963098 +0.j -0.10508999+0.j -0.0924313

In [ ]:
# Test RQIAPP with 1e-6
result = RQIAPP(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.48881554291037+0j)
Eigenvector: [0.08837012+0.j 0.10441203+0.j 0.10975581+0.j 0.10512379+0.j
 0.09593866+0.j 0.0950369 +0.j 0.10493569+0.j 0.09799041+0.j
 0.09799768+0.j 0.09722705+0.j 0.09886002+0.j 0.09245573+0.j
 0.101432  +0.j 0.09972556+0.j 0.09984459+0.j 0.09052967+0.j
 0.08862269+0.j 0.09206532+0.j 0.09732396+0.j 0.11534046+0.j
 0.10293406+0.j 0.10598362+0.j 0.08880361+0.j 0.09633757+0.j
 0.10660605+0.j 0.09291242+0.j 0.10249836+0.j 0.10772201+0.j
 0.09376061+0.j 0.10160191+0.j 0.10861449+0.j 0.09839231+0.j
 0.0927464 +0.j 0.10433217+0.j 0.10455709+0.j 0.09719659+0.j
 0.0938102 +0.j 0.10690809+0.j 0.10017636+0.j 0.09984143+0.j
 0.09813853+0.j 0.09753421+0.j 0.10444277+0.j 0.10495578+0.j
 0.09648881+0.j 0.09819335+0.j 0.10830215+0.j 0.10247659+0.j
 0.10436204+0.j 0.09805367+0.j 0.10896404+0.j 0.09926827+0.j
 0.09469168+0.j 0.10575157+0.j 0.09129068+0.j 0.09871294+0.j
 0.0963098 +0.j 0.10509   +0.j 0.09243133+0.j 0.08626875+0.j
 0.09801796+0.j 0.11475194+0.j 0.0980

In [ ]:
# Test RQI with 1e-8
result = RQI(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: None
Eigenvector: None
Iterations: 100
Condition Numbers: [2.13330702e+02 5.56337215e+04 4.25728493e+09 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16 5.29705127e+16 5.29705127e+16 5.29705127e+16
 5.29705127e+16

In [ ]:
# Test RQIAPP with 1e-8
result = RQIAPP(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.4888155022093+0j)
Eigenvector: [0.0883701 +0.j 0.10441202+0.j 0.10975579+0.j 0.1051238 +0.j
 0.09593866+0.j 0.09503692+0.j 0.10493568+0.j 0.09799041+0.j
 0.09799769+0.j 0.09722705+0.j 0.09886004+0.j 0.09245574+0.j
 0.10143199+0.j 0.09972556+0.j 0.09984457+0.j 0.09052966+0.j
 0.08862269+0.j 0.09206533+0.j 0.09732395+0.j 0.11534046+0.j
 0.10293405+0.j 0.10598362+0.j 0.0888036 +0.j 0.09633755+0.j
 0.10660604+0.j 0.09291242+0.j 0.10249836+0.j 0.10772202+0.j
 0.09376062+0.j 0.10160192+0.j 0.10861449+0.j 0.09839232+0.j
 0.0927464 +0.j 0.10433217+0.j 0.10455707+0.j 0.09719658+0.j
 0.09381019+0.j 0.1069081 +0.j 0.10017636+0.j 0.09984142+0.j
 0.09813852+0.j 0.09753421+0.j 0.10444276+0.j 0.10495578+0.j
 0.09648881+0.j 0.09819336+0.j 0.10830213+0.j 0.10247659+0.j
 0.10436205+0.j 0.09805367+0.j 0.10896404+0.j 0.09926826+0.j
 0.0946917 +0.j 0.10575159+0.j 0.09129067+0.j 0.09871292+0.j
 0.0963098 +0.j 0.10508999+0.j 0.09243134+0.j 0.08626875+0.j
 0.09801795+0.j 0.11475195+0.j 0.09801

In [ ]:
# Print actual eigenvalue/eigenvector
print(vals[0], vecs[:,0])

(49.48881550220932+0j) [-0.0883701 +0.j -0.10441202+0.j -0.10975579+0.j -0.1051238 +0.j
 -0.09593866+0.j -0.09503692+0.j -0.10493568+0.j -0.09799041+0.j
 -0.09799769+0.j -0.09722705+0.j -0.09886004+0.j -0.09245574+0.j
 -0.10143199+0.j -0.09972556+0.j -0.09984457+0.j -0.09052966+0.j
 -0.08862269+0.j -0.09206533+0.j -0.09732395+0.j -0.11534046+0.j
 -0.10293405+0.j -0.10598362+0.j -0.0888036 +0.j -0.09633755+0.j
 -0.10660604+0.j -0.09291242+0.j -0.10249836+0.j -0.10772202+0.j
 -0.09376062+0.j -0.10160192+0.j -0.10861449+0.j -0.09839232+0.j
 -0.0927464 +0.j -0.10433217+0.j -0.10455707+0.j -0.09719658+0.j
 -0.09381019+0.j -0.1069081 +0.j -0.10017636+0.j -0.09984142+0.j
 -0.09813852+0.j -0.09753421+0.j -0.10444276+0.j -0.10495578+0.j
 -0.09648881+0.j -0.09819336+0.j -0.10830213+0.j -0.10247659+0.j
 -0.10436205+0.j -0.09805367+0.j -0.10896404+0.j -0.09926826+0.j
 -0.0946917 +0.j -0.10575159+0.j -0.09129067+0.j -0.09871292+0.j
 -0.0963098 +0.j -0.10508999+0.j -0.09243134+0.j -0.08626875+0.j
 -

In [ ]:
# Function to run n rounds of RQI and RQIAPP
def runRQI(M, mu, ep, it, n):
  # Save number of iterations to arrays
  its = np.empty(0)
  itsAPP = np.empty(0)

  for i in range(n):
    # Save number of iterations required to the appropriate array
    its = np.append(its, RQI(M, mu, ep, it)[2])
    itsAPP = np.append(itsAPP, RQIAPP(M, mu, ep, it)[2])

  # Print iterations required each round
  print('Iterations required each round for RQI:\n', its)
  print('\nIterations required each round for RQIAPP:\n', itsAPP)

  # Get average iterations required by RQI
  index = np.argwhere(its==it)
  its = np.delete(its, index)
  if its.size==0:
    itsAvg = 0
  else:
    itsAvg = np.average(its)

  # Get average iterations required by RQIAPP
  index = np.argwhere(itsAPP==it)
  itsAPP = np.delete(itsAPP, index)
  if itsAPP.size==0:
    itsAPPAvg = 0
  else:
    itsAPPAvg = np.average(itsAPP)

  # Print average iterations required and number of errors
  print('\nAverage iterations required for RQI:\n', itsAvg)
  print('\nPercentage of times RQI converged:\n', str(its.size/n*100) + '%')
  print('\nAverage iterations required for RQIAPP:\n', itsAPPAvg)
  print('\nPercentage of times RQIAPP converged:\n', str(itsAPP.size/n*100) + '%')

In [ ]:
# Run 100 rounds with 1e-4
runRQI(M, mu, 1e-4, 20, 100)

Iterations required each round for RQI:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Iterations required each round for RQIAPP:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Average iterations required for RQI:
 2.0

Percentage of times RQI converged:
 100.0%

Average iterations required for RQIAPP:
 2.0

Percentage of times RQIAPP converged:
 100.0%


In [ ]:
# Run 100 rounds with 1e-6
runRQI(M, mu, 1e-6, 20, 100)

Iterations required each round for RQI:
 [ 3.  3.  3.  3.  3.  3.  3.  3.  3.  3. 20.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3. 20.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3. 20.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3. 20.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.]

Iterations required each round for RQIAPP:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Average iterations required for RQI:
 3.0

Percentage of times RQI converged:
 96.0%

Average iterations required for RQIAPP:
 2.0

Percentage of times RQIAPP converged:
 100.0%


In [ ]:
# Run 100 rounds with 1e-8
runRQI(M, mu, 1e-8, 20, 100)

Iterations required each round for RQI:
 [ 3. 20.  3.  3. 20. 20. 20.  3. 20.  3.  3. 20.  3. 20. 20.  3.  3. 20.
  3. 20. 20.  3.  3.  3. 20. 20. 20. 20. 20.  3. 20. 20.  3. 20. 20.  3.
 20. 20.  3.  3. 20. 20. 20.  3.  3. 20. 20. 20.  3.  3.  3. 20.  3.  3.
  3.  3. 20.  3. 20.  3. 20. 20. 20. 20.  3.  3. 20.  3. 20.  3. 20.  3.
  3.  3. 20.  3.  3.  3. 20. 20. 20.  3.  3.  3. 20. 20. 20. 20. 20. 20.
  3. 20.  3. 20. 20. 20.  3.  3. 20. 20.]

Iterations required each round for RQIAPP:
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 2. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 2. 3.]

Average iterations required for RQI:
 3.0

Percentage of times RQI converged:
 46.0%

Average iterations required for RQIAPP:
 2.96

Percentage of times RQIAPP converged:
 100.0%


In [ ]:
# Run 100 rounds with 1e-10
runRQI(M, mu, 1e-10, 20, 100)

Iterations required each round for RQI:
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]

Iterations required each round for RQIAPP:
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]

Average iterations required for RQI:
 0

Percentage of times RQI converged:
 0.0%

Average iterations required for RQIAPP:
 3.0

Percentage of times RQIAPP converged:
 100.0%
